In [1]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 

In [2]:
# !pip install tensorflow-recommenders==0.6.0 --user

In [3]:
TF_GPU_THREAD_MODE='gpu_private'

In [4]:
import json
import tensorflow as tf
import tensorflow_recommenders as tfrs

from google.cloud import storage

import numpy as np
import pickle as pkl
from pprint import pprint

from two_tower_src import two_tower as tt

2022-09-28 00:13:29.477868: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-28 00:13:30.120914: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38238 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:00:04.0, compute capability: 8.0


## Create Dataset for local training and testing

### Playlist dataset

In [5]:
train_files = []
for blob in client.list_blobs(f'{train_dir}', prefix=f'{train_dir_prefix}', delimiter="/"):
    train_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))

# OPTIMIZE DATA INPUT PIPELINE
train_dataset = tf.data.Dataset.from_tensor_slices(train_files)
train_dataset = train_dataset.interleave(
    lambda x: tf.data.TFRecordDataset(x),
    cycle_length=tf.data.AUTOTUNE, 
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=False
).map(
    tt.parse_tfrecord,
    num_parallel_calls=tf.data.AUTOTUNE,
).map(
    tt.return_padded_tensors,
    num_parallel_calls=tf.data.AUTOTUNE,)
# ).batch(
#     batch_size 
# ).prefetch(
#     tf.data.AUTOTUNE,
# )


valid_files = []
for blob in client.list_blobs(f'{valid_dir}', prefix=f'{valid_dir_prefix}', delimiter="/"):
    valid_files.append(blob.public_url.replace("https://storage.googleapis.com/", "gs://"))


# OPTIMIZE DATA INPUT PIPELINE
valid_dataset = tf.data.Dataset.from_tensor_slices(valid_files)
valid_dataset = valid_dataset.interleave(
    lambda x: tf.data.TFRecordDataset(x),
    cycle_length=tf.data.AUTOTUNE, 
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=False
).map(
    tt.parse_tfrecord,
    num_parallel_calls=tf.data.AUTOTUNE,
).map(
    tt.return_padded_tensors,
    num_parallel_calls=tf.data.AUTOTUNE,)
# ).batch(
#     batch_size
# ).prefetch(
#     tf.data.AUTOTUNE,
# )

In [6]:
from two_tower_src import preprocess as hash_data

valid_hashed = valid_dataset.map(hash_data.pre_hash_records)
train_hashed = train_dataset.map(hash_data.pre_hash_records)

In [7]:
candidate_hashed = tt.parsed_candidate_dataset.map(lambda x: 
                                            hash_data.pre_hash_records(x, candidate_mode=True))

In [ ]:
train_dir_hash = 'gs://spotify-beam-v3/v3/train-hashed/'

valid_dir_hash = 'gs://spotify-beam-v3/v3/valid-hashed/'

candidate_dir_hash = 'gs://spotify-beam-v3/v3/candidates-hashed/'

def custom_shard_func_train(element, n_shards=5000):
    x = element['track_uri_can'] % n_shards
    return(x)

def custom_shard_func_valid(element,  n_shards=510) -> tf.int64:
    x = tf.dtypes.cast(element['track_name_can'], tf.int64) % n_shards
    return(x)

def custom_shard_func_candidate(element,  n_shards=8) -> tf.int64:
    x = tf.dtypes.cast(element['track_name_can'], tf.int64) % n_shards
    return(x)

In [ ]:
tf.data.experimental.save(valid_hashed, valid_dir_hash, shard_func=custom_shard_func_valid)

In [ ]:
# tf.data.experimental.save(train_hashed, train_dir_hash, shard_func=custom_shard_func_train)

In [ ]:
tf.data.experimental.save(candidate_hashed, candidate_dir_hash, shard_func=custom_shard_func_candidate)